In [63]:
import nibabel as nib
import nilearn as nil
import numpy as np
import pandas as pd
import os
import os.path
import logging
from nilearn import plotting, image
import radiomics
from radiomics import featureextractor
from src.utils.data import getDataPandas

In [38]:
data = getDataPandas()
mask_path = 'bin/raal3.nii'
mask_template = nib.load(mask_path).get_fdata().astype(int)
mask_tags = {
    'lCAU': [75],
    'rCAU': [76],
    'lPUT': [77],
    'rPUT': [78],
    'rSN': [162, 164],
    'lSN': [161, 163]
    #'thalamus': [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
}
#mask[np.isin(mask_template, mask_tags['caudate nucleus'])] = 1
#mask[np.isin(mask_template, mask_tags['putamen'])] = 2
#mask[np.isin(mask_template, mask_tags['substantia nigra'])] = 3
#mask[np.isin(mask_template, mask_tags['thalamus'])] = 4

In [42]:
for tag, labels in mask_tags.items():
    mask = np.full(mask_template.shape, 0)
    for label in labels:
        mask[mask_template == label] = 1
    img = image.new_img_like(mask_path, mask)
    img.to_filename(os.path.join('bin', 'roi', tag + '.nii'))

In [48]:
radiomics.logger.setLevel(logging.ERROR)
extract = featureextractor.RadiomicsFeatureExtractor()
extract.loadParams(os.path.join('data', 'radiomic.yaml'))

In [160]:
def cal_radiomics(path):
    filtered_rst = {}
    for key in mask_tags.keys():
        rst = extract.execute(path, os.path.join('bin', 'roi', key + '.nii'))
        for k, v in rst.items():
            if ('firstorder' in k) or ('glcm' in k) or ('gldm' in k):
            #if ('firstorder' in k):
                filtered_rst[k.replace('original', key)] = v
    return filtered_rst

In [161]:
rsts = list(map(cal_radiomics, data['T1_GM_PATH']))

In [162]:
data_radiomic = pd.DataFrame(rsts)
data_radiomic = data_radiomic.astype(float)
data_radiomic['KEY'] = data['KEY']

In [163]:
import json
with open(os.path.join('data', 'json', 'data_radiomic.json'), 'w+', encoding="utf-8") as f:
        json.dump(data_radiomic.to_dict(orient='records'), f, ensure_ascii=False, indent=4)